# Guía mover datos Azure-BBD Local

In [1]:
! pip install pyodbc

En el código de a continuación movemos una base de datos en la nube a una base de datos local (SQL Server Local)

**Conceptos Importantes**
- Servidor: es una ordenador  donde se almacenan bases de datos. Puede estar en la nube o en tu propia computadora.
- Driver: programa que permite que python se comunique con una bbd.


In [2]:
import pyodbc
import pandas as pd
import numpy as np

#  Conexión a **Azure SQL**
AZURE_SERVER = 'uaxmathfis.database.windows.net' #servidor al  nos conectaremos en azure
AZURE_DATABASE = 'usecases' 
AZURE_DRIVER = '{ODBC Driver 17 for SQL Server}'  

azure_conn_str = f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};Authentication=ActiveDirectoryInteractive"

#  Conexión a **SQL Server LOCAL(Base datos PC)**
LOCAL_SERVER = 'localhost' #servidor local
LOCAL_DATABASE = 'dwh_case1' #base datos local
LOCAL_DRIVER = '{ODBC Driver 17 for SQL Server}'

# con este código nos estamos conectadn a sssm
local_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"

#  Consulta SQL en Azure SQL -Para extraer los daos de la nube
SQL_QUERY = """
SELECT
"""

# Nombre de la tabla en SQL Server Local donde se van a gaurdar los datos que extreamos meidante la consulta
NEW_TABLE_NAME = "DATAEX.FACT_SALES"

try:
    #  Conectar a Azure SQL
    print(f"Conectando a Azure SQL...")
    conn_azure = pyodbc.connect(azure_conn_str)
    
    # Ejecutar la consulta en Azure SQL y guardamos los datos en un dataframe
    print(f"Ejecutando consulta en Azure SQL...")
    df = pd.read_sql(SQL_QUERY, conn_azure) #dataframe con la tabla de la consulta

    if df.empty:
        print(f" La consulta no devolvió resultados. No se creará la tabla en SQL Server Local.")
    else:
        print(f"   - Datos extraídos: {df.shape[0]} filas") #rellenamos los valores con 0 para evitar errores



        #  Convertir NaN en columnas numéricas a 0
        df = df.fillna(0)

        #  Convertir valores numéricos problemáticos
        for col in df.select_dtypes(include=['float64']).columns:
            df[col] = df[col].astype(np.float32)  #Para ahorrar memoeia
        
        for col in df.select_dtypes(include=['int64']).columns:
            df[col] = df[col].astype(np.int32)  # Evitar valores fuera de rango
        
        #  Conectar a SQL Server Local
        print(f"Conectando a SQL Server Local...")
        conn_local = pyodbc.connect(local_conn_str)
        
        with conn_local.cursor() as cursor:
            # Eliminar la tabla si ya existe(esto eviat errores por duplicados)
            drop_table_sql = f"DROP TABLE IF EXISTS {NEW_TABLE_NAME}"
            cursor.execute(drop_table_sql)
            conn_local.commit()
            print(f"   - Tabla eliminada si existía.")

            #  Crear la tabla en SQL Server Local con tipos de datos ajustados 
            create_table_sql = f"""
            CREATE TABLE {NEW_TABLE_NAME} (
                {', '.join([
                    f'[{col}] FLOAT' if df[col].dtype == np.float32 
                    else f'[{col}] INT' if df[col].dtype == np.int32 
                    else f'[{col}] NVARCHAR(255)' for col in df.columns
                ])}
            );
            """
            cursor.execute(create_table_sql)
            conn_local.commit()
            print(f" Tabla {NEW_TABLE_NAME} creada correctamente en SQL Server Local.")

            # Insertar los datos en SQL Server Local
            placeholders = ', '.join(['?' for _ in df.columns]) #rellenar interrograciones para insterat datos
            insert_sql = f"INSERT INTO {NEW_TABLE_NAME} VALUES ({placeholders})"

            cursor.fast_executemany = True
            cursor.executemany(insert_sql, df.values.tolist()) #crea una lista de listas con los valores de df y los rellena en base de datos en loca
            conn_local.commit()

            print(f" {df.shape[0]} filas insertadas en {NEW_TABLE_NAME}.")

except Exception as e:
    print(f" Error: {e}")

finally:
    if 'conn_azure' in locals():
        conn_azure.close()
    if 'conn_local' in locals():
        conn_local.close()

print("\n ¡Proceso completado!")


Conectando a Azure SQL...
 Error: ('FA004', "[FA004] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Failed to authenticate the user '' in Active Directory (Authentication option is 'ActiveDirectoryInteractive').\r\nError code 0x4C7; state 10\r\nEl usuario ha cancelado la operación. (0) (SQLDriverConnect); [FA004] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Failed to authenticate the user '' in Active Directory (Authentication option is 'ActiveDirectoryInteractive').\r\nError code 0x4C7; state 10\r\nEl usuario ha cancelado la operación. (0)")

 ¡Proceso completado!


**Aclaraciones Del Código**
1. Los interrogantes se usan como marcadores de posiciones ,para posteriormente poder instar los valores d ela consulta en la tabla.(Evita la inyección en sql y hace la consulta más eficaz)
2. Un driver ODBC es un software que permite que aplicaciones de terceros se comuniquen con bases de datos. Funciona como un puente entre una aplicación y un sistema de gestión de bases de datos.
3. ¿ Por qué se insertan interrogaciones?

    SELECT * FROM usuarios WHERE username = 'admin' -- ' AND password = 'cualquiercosa'(se acepta a admin sin verificar contra)

    VS
    
    sql_query = "SELECT * FROM usuarios WHERE username = ? AND password = ?"



